# DEPRECATED

use the script duplex_metadata.py now

Notebook for analyzing the distribution of PCR duplicates in duplex sequencing runs

In [ ]:
import sys
import os
sys.path.append(os.getcwd() + '/../python_scripts') # this lets us import files in python_scripts (like gtools)
import gtools
if os.getcwd()[:8] != '/scratch': # switch to the scratch directory where all the data files are
    os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')

import pysam
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from collections import defaultdict
import random
import numpy as np
import statistics as stat
import matplotlib
import pandas as pd
import matplotlib.colors as colors
import multiprocessing
import pickle
plt.rcParams['svg.fonttype'] = 'none'
import itertools

In [ ]:
sample_names = ''
sample_names += 'Col-0-1 Col-0-2 Col-0-3 Col-0-4 Col-0-5 Col-0-6 Col-0-7 Col-0-8 '
# sample_names += 'chandler '
# sample_names += 'Col-0-swapped-1 Col-0-swapped-2 Col-0-swapped-3 Col-0-swapped-4 Col-0-swapped-5 Col-0-swapped-6 '
# sample_names += 'Ler-0-germline '
# sample_names += 'UVB-1 UVB-2 UVC-002 UVC-005 UVC-01 UVC-02 '
# sample_names += 'UVCnotime-0h UVCnotime-1h UVCnotreat-C1 UVCnotreat-C2 '
# sample_names += 'Col-0-bonus1 Col-0-bonus2 Col-0-bonus3 '
# sample_names += 'arp-1 arp-2 '
# sample_names += 'atx12r7-1 atx12r7-2 atx12r7-3 '
# sample_names += 'chr8-1 chr8-2 chr8-3 '
# sample_names += 'clf-1 clf-2 clf-3 '
# sample_names += 'ddm1-1 ddm1-2 ddm1-3 '
# sample_names += 'h2aw7-1 h2aw7-2 h2aw7-3 '
# sample_names += 'h2ax35-1 h2ax35-2 h2ax35-3 '
# sample_names += 'ku80-1 ku80-2 ku80-3 '
# sample_names += 'met1-epiRIL12-1 met1-epiRIL12-2 met1-epiRIL12-3 '
# sample_names += 'msh2-1 msh2-2 msh2-3 '
sample_names += 'msh6-1 msh6-2 msh6-3 '
# sample_names += 'nrpd1b-1 nrpd1b-2 nrpd1b-3 '
# sample_names += 'parp2-1 parp2-2 parp2-3 '
sample_names += 'poly-1 poly-2 poly-3 '
# sample_names += 'rad5a-1 rad5a-2 rad5a-3 '
# sample_names += 'rad7a-1 rad7a-2 rad7a-3 '
# sample_names += 'sdg8-1 sdg8-2 sdg8-3 '
# sample_names += 'suvh456-2 suvh456-3 '
# sample_names += 'xpg-1 xpg-2 xpg-3'
sample_names = sample_names.split()
# files = [f'data/align/big_{s}_merged.bam' for s in sample_names]
                
# sample_names = 'Col-0-1 Col-0-2 Col-0-3 Col-0-4 Col-0-5 Col-0-6 Col-0-7 Col-0-8 \
#                 Col-0-1_merged_01 Col-0-1_merged_02 Col-0-1_merged_04 Col-0-1_merged_06 Col-0-1_merged_08 \
#                 Col-0-2_merged_01 Col-0-2_merged_02 Col-0-2_merged_04 Col-0-2_merged_06 Col-0-2_merged_08 \
#                 Col-0-3_merged_01 Col-0-3_merged_02 Col-0-3_merged_04 Col-0-3_merged_06 Col-0-3_merged_08 \
#                 Col-0-4_merged_01 Col-0-4_merged_02 Col-0-4_merged_04 Col-0-4_merged_06 Col-0-4_merged_08 \
#                 Col-0-5_merged_01 Col-0-5_merged_02 Col-0-5_merged_04 Col-0-5_merged_06 Col-0-5_merged_08 \
#                 Col-0-6_merged_01 Col-0-6_merged_02 Col-0-6_merged_04 Col-0-6_merged_06 Col-0-6_merged_08 \
#                 Col-0-7_merged_01 Col-0-7_merged_02 Col-0-7_merged_04 Col-0-7_merged_06 Col-0-7_merged_08 \
#                 Col-0-8_merged_01 Col-0-8_merged_02 Col-0-8_merged_04 Col-0-8_merged_06 Col-0-8_merged_08 \
#                 Col-0-1_01frags Col-0-1_02frags Col-0-1_04frags Col-0-1_06frags Col-0-1_08frags \
#                 Col-0-2_01frags Col-0-2_02frags Col-0-2_04frags Col-0-2_06frags Col-0-2_08frags \
#                 Col-0-3_01frags Col-0-3_02frags Col-0-3_04frags Col-0-3_06frags Col-0-3_08frags \
#                 Col-0-4_01frags Col-0-4_02frags Col-0-4_04frags Col-0-4_06frags Col-0-4_08frags \
#                 Col-0-5_01frags Col-0-5_02frags Col-0-5_04frags Col-0-5_06frags Col-0-5_08frags \
#                 Col-0-6_01frags Col-0-6_02frags Col-0-6_04frags Col-0-6_06frags Col-0-6_08frags \
#                 Col-0-7_01frags Col-0-7_02frags Col-0-7_04frags Col-0-7_06frags Col-0-7_08frags \
#                 Col-0-8_01frags Col-0-8_02frags Col-0-8_04frags Col-0-8_06frags Col-0-8_08frags'.split()
files = []
for s in sample_names:
    if 'merged' in s:
        files.append(f'data/align/read_subsample/big_{s}.bam')
    elif 'frags' in s:
        files.append(f'data/align/fragment_subsample/big_{s}.bam')
    elif 'chandler' in s:
        files.append(f'data/align/chandler_KVKCS001D_0_filtered.bam')
    else:
        files.append(f'data/align/big_{s}_merged.bam')

genome_file = 'data/ref/arabidopsis_ref/ref.fa'
size_to_process = 1000000 # will only process reads on first chromosome up to this position

## Get coverage data of each fragment in the bam files

In [ ]:
# creates a pickled dictionary with key=source replicate (RG) value=[(chrom, start, tlen, umi, + strand pairs, - strand pairs)]
def get_frags(file, out, sub=2):
    fragments = defaultdict(lambda: [0, 0])
    aln = pysam.AlignmentFile(file, 'rb')
    total_reads = sum([x.total for x in aln.get_index_statistics()])
    rep_reads = defaultdict(lambda: 0)
    for read in tqdm(aln.fetch(contig='Chr1'), position=0, miniters=1000000, maxinterval=999):
        frag_start = read.reference_start
        tlen = abs(read.template_length) # this is non-inclusive
        if read.has_tag('RX'):
            umi = read.get_tag('RX')
            rep = umi.split('_')[-1]
        else:
            umi = read.get_tag('RG')
            rep = umi
        rep_reads[rep] += 1
        
        # thow out read pairs that don't align as expected, ignore read 2 so we don't double count pairs
        if read.is_unmapped or read.mate_is_unmapped or not (read.flag & 2) or read.is_reverse or read.mapping_quality < 10:
            continue
        
        if random.random() > sub:
            continue
        
        frag_strand = '+' if read.is_read1 else '-'
        
        if frag_strand == '+':
            fragments[(read.reference_name, frag_start, tlen, umi, rep)][0] += 1
        else:
            fragments[(read.reference_name, frag_start, tlen, umi, rep)][1] += 1
                
        if read.reference_start > size_to_process:
            break
    
    fragments = dict(fragments)
    frags = {rep:[] for rep in rep_reads}
    for frag in fragments:
        frags[frag[4]].append((frag[0], frag[1], frag[2], frag[3], fragments[frag][0], fragments[frag][1]))
    
    for rep in frags:
        frags[rep].sort() # the sort matters for the scrambled null section
    with open(out, 'wb') as fout:
        pickle.dump(frags, fout)
    
    # print(f'{file}: {total_reads} reads')
    # for rep in rep_reads:
    #     print(f'{int((rep_reads[rep] / sum(rep_reads.values())) * total_reads)} reads in replicate {rep}')

In [ ]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=8)
    processes = []
    for i in range(len(files)):
        processes.append(pool.apply_async(get_frags, (files[i], f'tmp/metadata_{sample_names[i]}_frags.pkl')))
    pool.close()
    pool.join()
    # check for errors
    for p in processes:
        p.get()

In [ ]:
frags = dict() # key: sample; value: [(chrom, frag_start, tlen, umi, + reads, - reads)]
for sample in tqdm(sample_names, total=len(files), position=0):
    with open(f'tmp/metadata_{sample}_frags.pkl', 'rb') as f:
        l = pickle.load(f)
        frags[sample] = l

In [ ]:
first_sample_frags = frags[sample_names[0]][list(frags[sample_names[0]])[0]]
strand_1 = np.array([x[4] for x in first_sample_frags])
strand_2 = np.array([x[5] for x in first_sample_frags])

avgs = []
for i in range(10):
    a = strand_1 * (strand_2 == i)
    avgs.append(sum(a) / np.count_nonzero(a))

fig, ax = plt.subplots()
ax.bar(range(10), avgs)
ax.set_xlabel('bottom strand coverage')
ax.set_ylabel('average top strand coverage')
ax.set_title('Duplex coverage correlates between strands')

# Calculate fragment conflict rate

In [ ]:
frags['Col-0-1']['big_Col-0_1a_filtered'][:10]

In [ ]:
conflict_rates = []
for i, sample in tqdm(enumerate(sample_names), total=len(sample_names)):
    
    # for each replicate, randomly sample half as many fragments from other replicates to compute a "half" conflict rate 
    # (the true conflict rate is calculated as 1 - (1-half conflict rate)^2)
    # the reason I calculate a "half" rate is that some samples only have 2 replicates, and if one has more fragments than the other, its 
    # "full" conflict rate can't be calculated, so I use the half as an extremely close approximation
    rates = []
    rep_frag_counts = []
    if len(frags[sample]) == 1 or 'swapped' in sample:
        conflict_rates.append(0)
        print(f'{sample} conflict rate of {conflict_rates[-1]}')
        continue
    
    for rep in frags[sample]:
        rep_frags = frags[sample][rep] # fragments from this rep
        other_frags = list(itertools.chain.from_iterable([frags[sample][r] for r in frags[sample] if r != rep])) # fragments from other reps
        
        # randomly select half as many fragments from other_frags as are in rep_frags
        ratio = len(rep_frags) / len(other_frags)
        if ratio > 2:
            continue
        other_subsample = {(x[0], x[1], x[2]):(x[4], x[5]) for x in other_frags if random.random() < 0.5 * ratio}
        
        # count the number of conflicts which make a fragment uncallable
        uncallable_conflicts = 0
        for frag in rep_frags:
            if (frag[0], frag[1], frag[2]) in other_subsample: # if there's a conflict
                # if the fragment doesn't contribute at least 76% of the top and bottom strand pairs, the conflict will be uncallable
                if frag[4] / max(1, (frag[4] + other_subsample[(frag[0], frag[1], frag[2])][0])) < 0.76 or \
                   frag[5] / max(1, (frag[5] + other_subsample[(frag[0], frag[1], frag[2])][1])) < 0.76:
                    uncallable_conflicts += 1
        conflict_rate = 1 - (1 - (uncallable_conflicts / len(rep_frags))) ** 2
        rates.append(conflict_rate)
        
        rep_frag_counts.append(sum([x[4] > 0 and x[5] > 0 for x in rep_frags])) # estimated number of callable fragments
    
    # calculate the sample conflict rate as the average across replicates, weighted by the number of fragments in the replicate
    conflict_rates.append(sum([rates[i] * rep_frag_counts[i] / sum(rep_frag_counts) for i in range(len(rates))]))
    print(f'{sample} conflict rate of {conflict_rates[-1]}')

In [ ]:
# make a best fit correlation between fragment number and conflict rate (only use samples where conflict rate could be estimated)
counts = []
rates = []
for i, sample in enumerate(sample_names):
    if conflict_rates[i] == 0:
        continue
    counts.append(sum([len(frags[sample][r]) for r in frags[sample]]))
    rates.append(conflict_rates[i])
counts += [0] * 10000 # my tiny brain can't figure out how to fix the intercept at 0, so I'm just adding a bunch of points at (0, 0)
rates += [0] * 10000
slope, intercept = np.polyfit(counts, rates, 1)
line_x = np.linspace(0, 5000000, 1000)
line_y = [x * slope + intercept for x in line_x]

In [ ]:
imputed_rates = []
for i, sample in enumerate(sample_names):
    if 'swapped' in sample:
        imputed_rates.append(0)
    elif conflict_rates[i] != 0:
        imputed_rates.append(conflict_rates[i])
    else:
        imputed_rates.append(sum([len(frags[sample][r]) for r in frags[sample]]) * slope + intercept)


In [ ]:
fig, ax = plt.subplots()
ax.scatter([sum([len(frags[s][r]) for r in frags[s]]) for s in sample_names], imputed_rates, color=['tab:blue' if x == 0 else 'tab:orange' for x in conflict_rates])
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.plot(line_x, line_y)
ax.set_xlim(xlim), ax.set_ylim(ylim)
ax.set_xlabel('Fragment number (in first Mb)')
ax.set_ylabel('Conflict rate')
ax.set_title('Conflict rate and fragment number')
s_blue = plt.scatter([], [], c='tab:blue')
s_orange = plt.scatter([], [], c='tab:orange')
ax.legend([s_orange, s_blue], ['calculated', 'imputed'])
fig.savefig('figs/conflict_rate_and_fragment_number.svg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print('conflict rates where rates for samples with no replicates are imputed:')
print([round(x, 5) for x in imputed_rates])

In [ ]:
print([round(x, 5) for x in imputed_rates[:8]])
print([round(x, 5) for x in imputed_rates[8:8 + (len(imputed_rates) - 8) // 2]])
print([round(x, 5) for x in imputed_rates[8 + (len(imputed_rates) - 8) // 2:]])

# Calculate genome coverage

In [ ]:
cycles = 300
metadata = []
for i, sample in enumerate(sample_names):
    total_reads = sum([x[4] + x[5] for x in frags[sample]])
    total_frags = len(frags[sample])
    callable_frags = len([frag for frag in frags[sample] if (frag[4] > 0 and frag[5] > 1) or (frag[4] > 1 and frag[5] > 0)])
    seq_eff = callable_frags / total_reads
    
    # metrics related to coverage, need to double check
    cov = total_reads * cycles / size_to_process
    opt_cov = cov * 0.107
    unique_cov = total_frags * cycles / size_to_process
    callable_cov = callable_frags * cycles / size_to_process
    
    metadata.append([file.split('/')[-1], total_frags, callable_frags, cov, opt_cov, unique_cov, callable_cov, seq_eff])
pd.DataFrame(metadata, columns='file total_frags callable_frags total_coverage optimal_coverage unique_coverage callable_coverage sequencing_efficiency'.split())

# Subsample to determine the optimal sequencing efficiency at perfect dilution

In [ ]:
sub_levels = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=8)
    processes = []
    for i, sample in enumerate(sample_names):
        for sub_level in sub_levels:
            processes.append(pool.apply_async(get_frags, (files[i], f'tmp/{sample}_sub_{sub_level}.pkl', sub_level)))
    pool.close()
    pool.join()

    # check for errors
    for p in processes:
        p.get()

In [ ]:
sub_frags = defaultdict(lambda: dict())
for i, sample in tqdm(enumerate(sample_names)):
    for sub_level in sub_levels:
        with open(f'tmp/{sample}_sub_{sub_level}.pkl', 'rb') as f:
            frags = pickle.load(f)
            for replicate in sorted(list(frags.keys())):
                sub_frags[sample + '_' + replicate][sub_level] = frags[replicate]


In [ ]:
# peak_eff_files = [defaultdict(lambda: ('', 0, 0, 0)) for x in range(3)] # indexed as [# reads needed to call - 1][index of original file]

subset_frags = {x:sub_frags[x] for x in list(sub_frags)}

# ############## Duplex-seq
cmap = plt.cm.tab20
fig, axs = plt.subplots(3, figsize=(len(subset_frags), 6), sharex=True, sharey=True)
for i, req_strand, req_total in zip(range(3), [1, 1, 2], [2, 3, 4]):
    seq_eff_overlap = []
    seq_eff_all = []
    for sample in subset_frags:
        for sub_level in sub_levels:
            total_reads = sum([x[4] + x[5] for x in subset_frags[sample][sub_level]])
            callable_frags_overlap = len([x for x in subset_frags[sample][sub_level] if x[4] >= req_strand and x[5] >= req_strand and x[4] + x[5] >= req_total])
            callable_frags_all = len([x for x in subset_frags[sample][sub_level] if x[4] >= 2 * req_strand and x[5] >= 2 * req_strand and x[4] + x[5] >= 2 * req_total])
            seq_eff_overlap.append(callable_frags_overlap / total_reads)
            seq_eff_all.append(callable_frags_all / total_reads)
    axs[i].bar(range(len(seq_eff_overlap)), seq_eff_overlap, color=[cmap((x // len(sub_levels)) % 2) for x in range(len(seq_eff_overlap))])
    axs[i].bar(range(len(seq_eff_all)), seq_eff_all, color=[cmap((x // len(sub_levels)) % 2 + 2) for x in range(len(seq_eff_all))])
    axs[i].set_title(f'Need >={req_strand * 2} reads per strand and >={req_total * 2} total to call', y=0.7)
axs[1].set_ylabel('Callable fragmens per aligned read pair')
axs[-1].set_xticks([len(sub_levels) // 2 + x * len(sub_levels) for x in range(len(subset_frags))])
axs[-1].set_xticklabels(subset_frags.keys(), rotation=45, ha='right')
axs[0].legend(['only read1/2 overlap is callable', 'all of fragment is callable'], bbox_to_anchor=(1, 1), loc='upper left')
fig.suptitle('Sequencing efficiency for subsampled libraries', y=0.95)
fig.savefig('figs/seq_efficiency_for_subsampled_libraries.png', dpi=300, bbox_inches='tight')
plt.show()

# ################# depreciated CODEC-seq
# fig, axs = plt.subplots(3, figsize=(18, 6), sharex=True, sharey=True)
# for i, req_strand, req_total in zip(range(3), [0, 0, 0], [1, 2, 3]):
#     seq_eff = []
#     for j, file in enumerate(sub_files):
#         total_reads = sum([x[4] + x[5] for x in sub_frags[file]])
#         callable_frags = len([x for x in sub_frags[file] if x[4] >= req_strand and x[5] >= req_strand and x[4] + x[5] >= req_total])
# #         print(file, total_reads, callable_frags)
#         seq_eff.append(callable_frags / total_reads)
#         if callable_frags / total_reads > peak_eff_files[i][j // len(sub_levels)][1]:
#             peak_eff_files[i][j // len(sub_levels)] = (file, callable_frags / total_reads, callable_frags, total_reads)
#     axs[i].bar(range(len(sub_files)), seq_eff, color=[cmap((x // len(sub_levels)) % 2) for x in range(len(sub_files))])
#     axs[i].set_title(f'Need >={req_strand} reads per strand and >={req_total} total to call', y=0.7)
# axs[1].set_ylabel('Callable fragmens per aligned read pair')
# axs[-1].set_xticks([len(sub_levels) // 2 + x * len(sub_levels) for x in range(len(files))])
# axs[-1].set_xticklabels(sample_names)
# fig.suptitle('Sequencing efficiency for subsampled libraries', y=0.95)
# fig.savefig('figs/seq_efficiency_for_subsampled_libraries.png', dpi=300, bbox_inches='tight')

# Check fragment lengths

In [ ]:
median_lens = []
for i, sample in enumerate(sample_names):
    values = [x[2] for x in frags[sample][list(frags[sample])[0]]]
    median_lens.append(stat.median(values))
    print(int(median_lens[-1]))

In [ ]:
fig, axs = plt.subplots(len(files), sharex=True)
fig.set_size_inches(12, 20)
for i, sample in tqdm(enumerate(sample_names)):
    axs[i].hist([x[2] for x in frags[sample][list(frags[sample])[0]]], bins=range(50, 2000, 10))
    axs[i].hist([x[2] for x in frags[sample][list(frags[sample])[0]] if x[4] > 0 and x[5] > 0], bins=range(50, 2000, 10))
    axs[i].set_title(f'{sample_names[i]} (median {int(median_lens[i])})', y=0.5, ha='left')
    # break
axs[-1].set_xlabel('Fragment size (bp)')
axs[len(files) // 2].set_ylabel('# fragments')
fig.suptitle('Duplex seq fragment lengths', y=0.91)
fig.savefig('figs/duplex_fragment_length.png', dpi=300)

# Visualize duplicate coverage per fragment

In [ ]:
# plot total coverage of each fragment to estimate the number of viable fragments in the library
fig, axs = plt.subplots(len(sample_names), sharex=True)
fig.set_size_inches(6, 8)
for i, sample in tqdm(enumerate(sample_names)):
    axs[i].hist([x[4] + x[5] for x in frags[sample]], bins=range(30))
    axs[i].hist([x[4] + x[5] for x in frags[sample] if x[4] > 0 and x[5] > 0], bins=range(30))
    axs[i].set_title(sample, y=0.5)
axs[-1].set_xlabel('# read pairs covering fragment')
axs[-1].set_xlabel('# read pairs covering fragment')
axs[len(files) // 2].set_ylabel('# fragments')
axs[0].legend(['Reads in 1 strand', 'Reads in both strands'])
fig.suptitle('Duplex seq fragment coverage', y=0.91)
fig.savefig('figs/duplex_fragment_total_coverage.png', dpi=300)

# Check if there's evidence that one strand of DNA is being lost

In [ ]:
sample_reps[-1]

In [ ]:
# plot fraction of two stranded support
sample_reps = []
for sample in sample_names:
    sample_reps += [(sample, rep) for rep in frags[sample]]
sample_reps = sample_reps[0:25:3]

fig, axs = plt.subplots(len(sample_reps), sharex=True)
fig.set_size_inches(6, 8)
for i, (sample, rep) in tqdm(enumerate(sample_reps)):
    fractions = []
    x_values = range(1, 20)
    for j in x_values:
        j_frags = [x for x in frags[sample][rep] if x[4] + x[5] == j]
        try:
            fractions.append(len([x for x in j_frags if x[4] != 0 and x[5] != 0]) / len(j_frags))
        except ZeroDivisionError:
            fractions.append(1)
    axs[i].plot(x_values, fractions)
    axs[i].plot(x_values, [(2**x - 2)/(2**x) for x in x_values])
    axs[i].set_title(sample + ' ' + rep, y=0.5)
axs[-1].legend(['Observed', 'Random sampling expectation'], loc='upper center', bbox_to_anchor=(0, -1, 0.5, 0))
axs[-1].set_xlabel('# read pairs covering fragment')
axs[len(axs) // 2].set_ylabel('Fraction of fragments with 2-stranded coverage')
fig.suptitle('Duplex seq fragment 2-stranded coverage', y=0.91)
fig.savefig('figs/duplex_fragment_2-stranded_coverage.png', dpi=300, bbox_inches='tight')

# Check if the "scrambled" null is feasible

In [ ]:
# iterate through the fragments of Col-0-1 and count how many reads are in Col-0-{2-8} with the same start and tlen
idxs = [0] * 8
self_reads = [] # number of reads in Col-0-1 for each fragment
other_reads = [] # number of reads in Col-0-{2-8} for each fragment
num_sources = [] # number other samples with a matching fragment
last_span = ('', 0, 0)
for i, frag in tqdm(enumerate(frags[sample_names[0]])): # for each fragment in Col-0-1
    span = (frag[0], frag[1], frag[2]) # chrom, start, tlen (don't consider umi)
    if span == last_span: # don't double count if there's more than one fragment in Col-0-1 with the same start and tlen
        continue
    last_span = span
    
    others = 0
    sources = 0 
    for j in range(1, 8): # for Col-0-{2-8}
        # iterate through fragments until past the current Col-0-1 fragment
        while idxs[j] < len(frags[sample_names[j]]): # don't go past the list end
            comp_frag = frags[sample_names[j]][idxs[j]]
            comp_span = (comp_frag[0], comp_frag[1], comp_frag[2])
            if comp_span > span:
                break
            if comp_span == span: # if the fragment matches
                others += comp_frag[4] + comp_frag[5]
                sources += 1
            idxs[j] += 1

    self_reads.append(frag[4] + frag[5])
    other_reads.append(others)
    num_sources.append(sources)
    
read_sources = np.zeros((3, len(self_reads))) # indexed as read_sources[0 (self)/1 (other)][fragment index] = number of reads
read_sources[0] = self_reads
read_sources[1] = other_reads
read_sources[2] = num_sources

In [ ]:
# number of Col-0-1 fragments with at least one matching fragment in another sample
np.count_nonzero(read_sources[1]), len(read_sources[1])

In [ ]:
max_sources = 10
bins = [x / 10 for x in range(12)]
bars = np.zeros((max_sources, len(bins)))
frac_in_self = read_sources[0] / (read_sources[0] + read_sources[1])
for i, b in enumerate(bins[:-1]):
    in_bin = np.logical_and(frac_in_self >= b, frac_in_self < bins[i + 1])
    column = np.zeros(max_sources)
    for s in range(max_sources):
        column[s] = np.count_nonzero(np.logical_and(in_bin, read_sources[2] <= s))
    bars[:, i] = column

In [ ]:
fig, ax = plt.subplots()
for i in range(max_sources - 1, -1, -1):
    ax.bar([x + (bins[1] - bins[0]) / 2 for x in bins], bars[i], width=bins[1] - bins[0])
ax.legend(range(max_sources - 1, -1, -1), title='Found in X other samples', ncol=2, loc='upper left', bbox_to_anchor=(1, 1))
ax.set_xlabel('reads in Col-0-1 / reads in all Col-0 samples')
ax.set_ylabel('# fragments in Col-0-1')
ax.set_title('Fragments with copies in other samples')
fig.savefig('figs/fragments_with_copies_in_other_samples.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot factors of PCR bias

In [ ]:
bins = list(range(50, 300, 10))
bin_start = 40
dups_by_size = defaultdict(lambda: [])
sterrs_by_size = defaultdict(lambda: [])
counts_by_size = defaultdict(lambda: [])
for sample in sample_names:
    last_bin = bin_start
    for b in tqdm(bins, position=0):
        dup_counts = [x[4] + x[5] for x in frags[sample] if x[2] < b and x[2] >= last_bin]
        dups_by_size[sample].append(sum(dup_counts) / len(dup_counts) if len(dup_counts) > 0 else 0)
        if len(dup_counts) > 1:
            sterrs_by_size[sample].append(stat.stdev(dup_counts) / (len(dup_counts) ** (1/2)))
        else:
            sterrs_by_size[sample].append(0)
        counts_by_size[sample].append(len(dup_counts))
        last_bin = b
    max_count = max(counts_by_size[sample])
    counts_by_size[sample] = [x / max_count for x in counts_by_size[sample]]

In [ ]:
fig, axs = plt.subplots(len(files), sharex=True, figsize=(12, 8))
cmap = matplotlib.cm.get_cmap('viridis')
for i, sample in enumerate(sample_names):
    axs[i].bar(range(len(bins)), dups_by_size[sample], yerr=sterrs_by_size[sample], color=[cmap(x) for x in counts_by_size[sample]])
    axs[i].set_title(sample_names[i], y=0.7, x=0.1)
axs[-1].set_xticks(range(0, len(bins), 5))
axs[-1].set_xticklabels(bins[0::5])
axs[-1].set_xlabel('Fragment size')
axs[len(files) // 2].set_ylabel('Average PCR duplicates')
fig.suptitle('PCR bias by fragment size', y=0.92)
fig.savefig('figs/PCR_bias_by_fragment_size.png', dpi=300)
plt.show()

In [ ]:
genome = dict()
with open(genome_file, 'r') as f:
    lines = f.read().split('>')[1:]
    for l in lines:
        genome[l.split(None, 1)[0]] = l.split('\n', 1)[1].replace('\n', '')

In [ ]:
bins = [x / 100 for x in range(5, 105, 5)]
bin_start = 0
dups_by_gc = defaultdict(lambda: [])
callable_by_gc = defaultdict(lambda: [])
sterrs_by_gc = defaultdict(lambda: [])
counts_by_gc = defaultdict(lambda: [])
for sample in sample_names:
    last_bin = bin_start
    for b in tqdm(bins, position=0):
        dup_counts = []
        callable_counts = [0, 0]
        for x in frags[sample]:
            sequence = genome[x[0]][x[1]:x[1] + x[2]]
            gc = (sequence.count('C') + sequence.count('G')) / len(sequence)
            if gc < b and gc >= last_bin:
                dup_counts.append(x[4] + x[5])
                if x[4] > 0 and x[5] > 0:
                    callable_counts[0] += 1
                callable_counts[1] += 1
        
        if len(dup_counts) > 1:
            dups_by_gc[sample].append(sum(dup_counts) / len(dup_counts))
            callable_by_gc[sample].append(callable_counts[0] / callable_counts[1])
            sterrs_by_gc[sample].append(stat.stdev(dup_counts) / (len(dup_counts) ** (1/2)))
        else:
            dups_by_gc[sample].append(0)
            callable_by_gc[sample].append(0)
            sterrs_by_gc[sample].append(0)
        counts_by_gc[sample].append(len(dup_counts))
        last_bin = b
    
    max_count = max(counts_by_gc[sample])
    counts_by_gc[sample] = [x / max_count for x in counts_by_gc[sample]]

In [ ]:
fig, axs = plt.subplots(len(files), sharex=True, figsize=(12, 8))
cmap = matplotlib.cm.get_cmap('viridis')
for i, sample in enumerate(sample_names):
    axs[i].bar(range(len(bins)), dups_by_gc[sample], yerr=sterrs_by_gc[sample], color=[cmap(x) for x in counts_by_gc[sample]])
    axs[i].set_title(sample_names[i], y=0.7, x=0.9)
axs[-1].set_xticks(range(0, len(bins), 2))
bin_size = bins[1] - bins[0]
axs[-1].set_xticklabels([round(b - bin_size / 2, 3) for b in bins[::2]])
axs[-1].set_xlabel('GC content')
axs[len(files) // 2].set_ylabel('Average PCR duplicates')
fig.suptitle('PCR bias by GC content', y=0.92)
fig.savefig('figs/PCR_bias_by_GC_content.png', dpi=300)
plt.show()

In [ ]:
gc = sum([genome[chrom].count('C') + genome[chrom].count('G') for chrom in genome])
print(f'genomic gc content: {gc / sum([len(genome[chrom]) for chrom in genome])}')